# Imports

In [5]:
import yfinance as yf
import pandas as pd
import requests_cache
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_community.document_loaders import PubMedLoader
from langchain_community.retrievers import PubMedRetriever
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain

import yake
import pandas as pd

In [2]:
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(5, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

# Finance Data

In [78]:
def get_stock_info(ticker: str, length: int):
    assert length in [1, 7, 30, 90, 365]
    lengthInt = [1, 7, 30, 90, 365]
    lengthStr = ["day", "week", "month", "quarter", "year"]
    try:
        stock = yf.Ticker(ticker, session=session)
        
    except:
        return None
    end_date = pd.Timestamp.today().strftime('%Y-%m-%d')
    start_date = (pd.Timestamp.today() - pd.Timedelta(days=length)).strftime('%Y-%m-%d')
    data  = {
        # Descriptive
        "ticker" : ticker,
        "period" : lengthStr[lengthInt.index(length)],
        "name": stock.info["shortName"] if "shortName" in stock.info else ticker,
        "indicators":
            {
                "dividentYield" : stock.info["dividendYield"],
                "beta" : stock.info["beta"],
                "PE": stock.info["trailingPE"],
                "fiftyDayAverage": stock.info["fiftyDayAverage"],
                "twoHundredDayAverage": stock.info["twoHundredDayAverage"],
                "shortRatio": stock.info["shortRatio"],
                "trailingEps": stock.info["trailingEps"],
                "ebitda": stock.info["ebitda"],
                "currentRatio": stock.info["currentRatio"],
                "totalRevenue": stock.info["totalRevenue"],
                "freeCashflow": stock.info["freeCashflow"],
                "operatingMargins": stock.info["operatingMargins"],
                "profitMargins": stock.info["profitMargins"],
                "earningsGrowth": stock.info["earningsGrowth"],
            },
        "website": stock.info["website"],
        "sector": stock.info["sector"],
        "description": stock.info["longBusinessSummary"],
        "marketCap" : stock.info["marketCap"],
        "currency": stock.info["currency"],
        "info": stock.info,
        "price_history" : stock.history(start=start_date, end=end_date) if length <= 90 else stock.history(start=start_date, end=end_date).ilo[::5],
        "income_statement": stock.income_stmt.iloc[:, :1],
        "balance_sheet": stock.balance_sheet.iloc[:20, :1],
        "cash_flow": stock.cashflow.iloc[:20, :1],
        "recommendations": stock.recommendations,
        "news": stock.news,
            }
    return data

In [74]:
temp_data = get_stock_info("AAPL", 7)

In [75]:
temp_data

{'ticker': 'AAPL',
 'period': 'week',
 'name': 'Apple Inc.',
 'indicators': {'dividentYield': 0.0056,
  'beta': 1.289,
  'PE': 26.668741,
  'fiftyDayAverage': 181.2792,
  'twoHundredDayAverage': 183.52095,
  'shortRatio': 1.64,
  'trailingEps': 6.43,
  'ebitda': 130108997632,
  'currentRatio': 1.073,
  'totalRevenue': 385706000384,
  'freeCashflow': 86563127296,
  'operatingMargins': 0.33764,
  'profitMargins': 0.26163,
  'earningsGrowth': 0.16},
 'website': 'https://www.apple.com',
 'sector': 'Technology',
 'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store 

# Finance LLM   

In [70]:
# https://ollama.com/hemanth/financialanalyst
#### SETUP #####
model = "gemma:2b"
numDocs = 5
context = []

In [76]:
# period = day, month, year
# ticker, name, marketCap, priceHistory, incomeStatement, balanceSheet, news, recommendations
prompt = ChatPromptTemplate.from_template(
"""
[INST] Write a brief summary outlining the key points and financial indicators for the stock ticker focused on the past {period}. 
Make sure to higlight any specific outlier data points and provide a brief analysis of the company's financial health and future outlook, highlighting specific data. 
Write as if it was a report to an investor interested in this company, and to recommend whether this stock seems like a good buy or sell. [/INST]
 
Stock ticker: {ticker}
Company name: {name}
Market cap: {marketCap}
Period: {period}

<Price History>
{price_history}
</Price History>

<Indicators>
{indicators}
</Indicators>

<Income Statement>
{income_statement}
</Income Statement>

<Relevant News>
{news}
</Relevant News>

<Output Template>
Key Points:
[bulleted list]

Financial Analysis:
[bulleted list]

Future Outlook / Recommendations:
[bulleted list]

Summary: 
[paragraph]
</Output Template>
""")

"""<Income Statement>
{income_statement}
</Income Statement>

<Balance Sheet>
{balance_sheet}
</Balance Sheet>

<Relevant News>
{news}
</Relevant News>

<Recommendations>
{recommendations}
</Recommendations>
    """
    
llm = Ollama(model=model)
chain = prompt | llm

In [77]:
# ticker, name, marketCap, priceHistory, incomeStatement, balanceSheet, news, recommendations
data = {
    "ticker": temp_data["ticker"],
    "period": temp_data["period"],
    "name": temp_data["name"],
    "marketCap": temp_data["marketCap"],
    "price_history": temp_data["price_history"],
    "indicators": temp_data["indicators"],
    "income_statement": temp_data["income_statement"],
    "balance_sheet": temp_data["balance_sheet"],
    "news": [x["title"] for x in temp_data["news"]],
    "recommendations": temp_data["recommendations"][:int(len(temp_data["recommendations"])/2)],
}
response = chain.invoke(data)

"## Key Points:\n\n* The stock price fluctuated significantly in the past week, with a high of 173.60 and a low of 169.44.\n* The closing price on Friday was 170.85, which is above the average daily price of 169.22.\n* The dividend yield is 0.0056, and the payout ratio is 107%.\n* The beta is 1.289, indicating a high degree of correlation with the overall stock market.\n* The price-to-earnings ratio is 26.66, which is above the average ratio of 20.05.\n* The company has a strong balance sheet with a low debt-to-equity ratio of 0.33.\n* The company has a positive net income and a high operating margin of 33.7%.\n* The company has a diverse product portfolio with a strong market position in the smartphone market.\n\n## Financial Analysis:\n\n* The company's revenue and profit margins are both high, indicating strong operational performance.\n* The company has a high dividend payout ratio, which could make it a good option for income investors.\n* The company's valuation is slightly highe

In [3]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
cred = credentials.Certificate('finsight-5cae6-firebase-adminsdk-8vm5q-04e459691c.json')
fb_app = firebase_admin.initialize_app(cred)
db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [14]:
ticker = "MSFT"
length = 7
lengthInt = [1, 7, 30, 90, 365]
lengthStr = ["day", "week", "month", "quarter", "year"]
typePeriod = lengthStr[lengthInt.index(length)]
currentDay = pd.Timestamp.today().day
currentMonth = pd.Timestamp.today().month
currentYear = pd.Timestamp.today().year
docName = ""
llm_cache_ref = db.collection("llm-cache")
if typePeriod == "day":
    docName = f"{ticker}-{currentMonth}-{currentDay}-{currentYear}-day"
elif typePeriod == "week":
    docName = f"{ticker}-{currentMonth}-{int(currentDay / 7)}-{currentYear}"
elif typePeriod == "month":
    docName = f"{ticker}-{currentMonth}-{currentYear}"
elif typePeriod == "quarter":
    docName = f"{ticker}-{int(currentMonth / 3)}-{currentYear}"
elif typePeriod == "year":
    docName = f"{ticker}-{currentYear}"
if (llm_cache_ref.document(docName).get().exists):
    temp_dict = llm_cache_ref.document(docName).get().to_dict()
    response = temp_dict["response"]
    description = temp_dict["description"]
    name = temp_dict["name"]
    sector = temp_dict["sector"]
else:
    response = "tester response"
    description = "tester description"
    name = "tester name"
    sector = "tester sector"
    llm_cache_ref.document(docName).set(
        {"response" : response,
         "description" : description,
         "name" : name,
         "sector" : sector}
    )